In [1]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("unicamp-dl/ptt5-base-portuguese-vocab")

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [3]:
# solve GPTJ import
from transformers import GPTJConfig, GPTJForCausalLM
import json

json_file = open('model/config.json')
config = json.load(json_file)

config = GPTJConfig(**config)
model = GPTJForCausalLM(config)

ImportError: cannot import name 'GPTJConfig' from 'transformers' (unknown location)

In [4]:
from datasets import load_dataset

dataset = load_dataset("assin2")

Found cached dataset assin2 (C:/Users/Matheus/.cache/huggingface/datasets/assin2/default/1.0.0/8467f7acbda82f62ab960ca869dc1e96350e0e103a1ef7eaa43bbee530b80061)
100%|██████████| 3/3 [00:00<00:00, 750.68it/s]


In [5]:
train_data = dataset['train']['premise']

In [6]:
train_data_tokenized = tokenizer.batch_encode_plus(
    train_data,
    max_length=model.config.n_positions,
    truncation=True,
    padding="max_length",
    return_tensors="pt",
)

In [7]:
import torch

# output
train_data_tokenized['label'] = train_data_tokenized['input_ids'][:, 1:13]
# train_data_tokenized['label'] = torch.cat((train_data_tokenized['label'], torch.ones(train_data_tokenized['label'].shape[0], 1, dtype=torch.int64)*tokenizer.eos_token_id), dim=1)

# input
train_data_tokenized['input_ids'] = train_data_tokenized['input_ids'][:, 0:12]
# train_data_tokenized['input_ids'] = torch.cat((train_data_tokenized['input_ids'], torch.ones(train_data_tokenized['input_ids'].shape[0], 1, dtype=torch.int64)*tokenizer.eos_token_id), dim=1)
train_data_tokenized['attention_mask'] = train_data_tokenized['attention_mask'][:, 0:12]
# train_data_tokenized['attention_mask'] = torch.cat((train_data_tokenized['attention_mask'], torch.ones(train_data_tokenized['attention_mask'].shape[0], 1, dtype=torch.int64)), dim=1)

### training loop

In [8]:
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm

class GPT2Dataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

train_dataset = GPT2Dataset(train_data_tokenized)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=False)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [9]:
# import cross entropy
_batch = next(iter(train_loader))

for epoch in range(3):
    model.train()
    loop = tqdm(train_loader)
    for _ in loop:
        input_ids = _batch['input_ids'].to(device)
        attention_mask = _batch['attention_mask'].to(device)
        labels = _batch['label'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

C:\Users\Matheus\AppData\Local\Temp\ipykernel_9812\602704409.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 1:   8%|▊         | 246/3250 [00:02<00:31, 95.48it/s, loss=0.000111] 


KeyboardInterrupt: 

In [10]:
# _text = 'Uma criança risonha está'
# _input = tokenizer.batch_encode_plus([_text], return_tensors="pt",)['input_ids']
# _input = torch.cat((_input, torch.ones(_input.shape[0], 1, dtype=torch.int64)*tokenizer.eos_token_id), dim=1).to(device)
_input = _batch['input_ids'].to(device)
output = model.generate(input_ids=_input, max_length=20)# , num_beams=5, no_repeat_ngram_size=2)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


Uma criança risonha está segurando uma pistola de água e e e e e e e e e


In [ ]:
tokenizer.decode(_batch['input_ids'][0], skip_special_tokens=True)

'Uma criança risonha está segurando uma pistola de água e'

In [ ]:
tokenizer.decode(_batch['label'][0], skip_special_tokens=True)

'criança risonha está segurando uma pistola de água e sendo'

In [ ]:
from torch.nn import CrossEntropyLoss

loss_fct = CrossEntropyLoss()

for epoch in range(3):
    model.train()
    loop = tqdm(train_loader)
    for _batch in loop:
        input_ids = _batch['input_ids'].to(device)
        attention_mask = _batch['attention_mask'].to(device)
        labels = _batch['label'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = loss_fct(outputs[1], labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())